In [8]:
#Rastgele bir film seçmek için gereken rastgele modülünü ekler.
import random
#Kullanıcının bir filmi beğendiğinde IMDb sayfasını açmak için gereken webbrowser modülünü ekler.
import webbrowser
#IMDb veri tabanındaki filmlere erişmek için gerekli olan IMDb modülünü ekler.
from imdb import IMDb

print('''__          __  _                                __  __            _       _____            __ _                
 \ \        / / | |                              |  \/  |          (_)     / ____|          / _| |               
  \ \  /\  / /__| | ___ ___  _ __ ___   ___      | \  / | _____   ___  ___| |     _ __ __ _| |_| |_ ___ _ __ ___ 
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \     | |\/| |/ _ \ \ / / |/ _ \ |    | '__/ _` |  _| __/ _ \ '__/ __|
    \  /\  /  __/ | (_| (_) | | | | | |  __/  _  | |  | | (_) \ V /| |  __/ |____| | | (_| | | | ||  __/ |  \__ \ 
     \/  \/ \___|_|\___\___/|_| |_| |_|\___| ( ) |_|  |_|\___/ \_/ |_|\___|\_____|_|  \__,_|_|  \__\___|_|  |___/
                                             |/                                                                 ''') 

# IMDbPY kütüphanesini kullanarak veri setini indir
# IMDb sınıfından bir örnek oluşturarak IMDb veri tabanına bağlanır.
ia = IMDb()
 ## IMDb'deki en iyi 250 filmi top_movies adlı değişkene atar.
 ##  get_top250_movies() :IMDbPY kütüphanesi tarafından sağlanan özel bir yöntemdir ve en iyi 250 filmin tam listesini IMDb'den alır
 ## Başka örnek verelim->> popular_movies = ia.get_popular_movies(year=2021, count=100) ise get_popular_movies() yöntemini çağırarak, 2021 yılında en popüler 100 film listesini alır ve bu listeyi popular_movies adlı bir değişkene atar.
top_movies = ia.get_top250_movies()
 ##En iyi 500 filmi best_movies adlı değişkene atar.
best_movies = top_movies[:500]

# Kullanıcının beğenmediği özellikleri saklamak için bir liste oluştur
disliked_directors = []
disliked_actors = []
disliked_genres = []
disliked_years = []
disliked_countries = []
disliked_rating =[]

while True:
    try:
        # best_movies listesinden rastgele bir film seçer.
        random_movie = random.choice(best_movies)
        # Seçilen filmin özelliklerini IMDb'den alır.
        movie = ia.get_movie(random_movie.getID())
        

        # Eğer filmin yönetmeni kullanıcının beğenmediği yönetmenler listesindeyse, başka bir film seç
        if movie['director'][0]['name'] in disliked_directors:
            continue
        # Eğer filmin oyuncularından biri kullanıcının beğenmediği oyuncular listesindeyse, başka bir film seç
        if any(actor['name'] in disliked_actors for actor in movie['cast']):
            continue
        # Eğer filmin türü kullanıcının beğenmediği türler listesindeyse, başka bir film seç
        if movie['genres'] in disliked_genres:
            continue
        # Eğer filmin ülke kategorisini neden olarak seçtiyse, başka bir film seç.
        if movie['countries'] in disliked_countries:
            continue
        # Eğer filmin yılı kullanıcının beğenmediği yıl istesindeyse, başka bir film seç
        if movie['year'] in disliked_years:
            continue
        # Eğer filmin IMDB puanını beğenmediğiyse, başka film seç
        if movie['rating'] in disliked_rating:
            continue

        # Filmin adı, yönetmeni, çıkış yılı, en önemli oyuncuları, türü ve IMDb puanını yazdır
        print("Önerilen film: ", movie['title'])
        print("Yönetmen: ", movie['director'][0]['name'])
        print("Çıkış Yılı: ", movie['year'])
        print("En İyi Oyuncular: ", ', '.join(actor['name'] for actor in movie['cast'][:5]))
        print("Tür: ", ', '.join(movie['genres']))
        print("IMDb Puanı: ", movie['rating'])
        print("Ülke:", movie['countries'])
        print("*****************")

        # Kullanıcının filmi beğenip beğenmediğini sor
        liked_movie = input("Bu filmi beğendiniz mi? (evet/hayır/exit): ")

        # Kullanıcının cevabına göre bir link ver
        if liked_movie.lower() == "evet":
        # Örnek Url: https://www.imdb.com/title/tt0137523/... # tt'den sonraki kısmı getID fonksiyonu ile alıyoruz.
            webbrowser.open_new_tab("https://www.imdb.com/title/tt" + str(movie.getID()))
            print("İyi seyirler!")
            break  # döngüyü kır ve programı sonlandır

        elif liked_movie.lower() == "hayır":
            print("Neden beğenmediniz?")
            print("1. IMDb Puanı")
            print("2. Yönetmen")
            print("3. Oyuncu Kadrosu")
            print("4. Filmin Türü")
            print("5. Çıkış Yılı")
            print("6. Ülke")
            print("*****************")
            reason = input("Seçiminiz (1/2/3/4/5/6): ")

            # Eğer kullanıcı neden olarak yönetmeni seçerse, yönetmeni beğenmediği yönetmenler listesine ekle
            if reason == "1":
               rating= float(input("Filmin güncel puanını giriniz."))
               disliked_rating.append(movie["rating"])
               print(f"Seçiminiz alındı. {rating} puanlı filmler önerilmeyecek.")
               
            # Eğer kullanıcı neden olarak aktörleri seçerse, aktörleri beğenmediği aktör listesine ekle
            elif reason == "2":
                disliked_directors.append(movie['director'][0]['name'])
                print(f" Seçiminiz alındı {movie['director'][0]['name']} adlı bu yönetmenin filmleri bir daha önerilmeyecek.")

            # Eğer kullanıcı neden olarak oyuncu kadrosunu seçerse, en iyi oyuncular listesindeki elemanları seçenek olarak sun ve kullanıcı o seçeneklerden birini seçtikten sonra o oyuncaya ait hiç bir filmi önerme.
            elif reason == "3":
                actors = [actor['name'] for actor in movie['cast'][:5]]
                print("Hangi oyuncudan dolayı beğenmediniz?")
                for index, actor in enumerate(actors):
                    print(f"{index + 1}. {actor}")
                disliked_actor_index = int(input(f"Seçiminiz (1-{len(actors)}): ")) - 1
                disliked_actor = actors[disliked_actor_index]
                disliked_actors.append(disliked_actor)
                print(f" Seçiminiz alındı {disliked_actor} adlı bu oyuncunun filmleri bir daha önerilmeyecek.")
                print("***************")

            #Eğer kullanıcı neden olarak film türü seçerse, film türünü beğenmediği tür listesine ekle.
            elif reason == "4":
                genre = movie['genres']
                print("Hangi film türünü beğenmediniz?")
                for index, genre_name in enumerate(genre):
                    print(f"{index + 1}. {genre_name}")
                disliked_genre_index = int(input(f"Seçiminiz (1-{len(genre)}): ")) - 1
                disliked_genre = genre[disliked_genre_index]
                disliked_genres.append(disliked_genre)
                print(f" Seçiminiz alındı {disliked_genre} adlı bu film türü bir daha önerilmeyecek.")
                print("***************")

            #Eğer kullanıcı neden olarak çıkış yılı seçerse, film türünü beğenmediği çıkış yılı listesine ekle.
            elif reason == "5":
                years = movie['year']
                disliked_years.append(movie['year'])
                print(f"Seçiminiz alındı {movie['year']} çıkış yılı olan filmler önerilmeyecektir.")

            #Eğer kullanıcı neden olarak ülke seçerse, beğenmediği ülke listesine ekle.
            elif reason == "6":
                countries = movie['countries']
                disliked_countries.append(movie['countries'])
                print(f"Seçiminiz alındı {movie['countries']} ülke yapımı olan filmler önerilmeyecektir.")    
        # Kullanıcı programdan çıkış yapmak isterse, q'ya tıklar.
        elif liked_movie.lower() == "exit":
            print("Program sonlandı.")
            break
    # Hata yapısı.
    except ValueError as e:
        print("Lütfen doğru girdiğinizden emin olunuz.")
        break
    

__          __  _                                __  __            _       _____            __ _                
 \ \        / / | |                              |  \/  |          (_)     / ____|          / _| |               
  \ \  /\  / /__| | ___ ___  _ __ ___   ___      | \  / | _____   ___  ___| |     _ __ __ _| |_| |_ ___ _ __ ___ 
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \     | |\/| |/ _ \ \ / / |/ _ \ |    | '__/ _` |  _| __/ _ \ '__/ __|
    \  /\  /  __/ | (_| (_) | | | | | |  __/  _  | |  | | (_) \ V /| |  __/ |____| | | (_| | | | ||  __/ |  \__ \ 
     \/  \/ \___|_|\___\___/|_| |_| |_|\___| ( ) |_|  |_|\___/ \_/ |_|\___|\_____|_|  \__,_|_|  \__\___|_|  |___/
                                             |/                                                                 
Önerilen film:  Cinema Paradiso
Yönetmen:  Giuseppe Tornatore
Çıkış Yılı:  1988
En İyi Oyuncular:  Antonella Attili, Enzo Cannavale, Isa Danieli, Leo Gullotta, Marco Leonardi
Tür:  Drama, Romance
IMDb Pu